In [ ]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from IPython.display import Markdown
from plotly.express.colors import qualitative as qual_colours
from pathlib import Path
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Markdown

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, plot_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, PANEL_SUBTITLES, plot_3d_spaghetti, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.utils import get_adjust_idata_index, adjust_summary_cols

In [ ]:
case_data = pd.read_csv(r"C:\Users\ehug0006\emu\aust-renewal\aust_renewal\data\WHO-COVID-19-global-data_21_8_24.csv")
seroprev_data = pd.read_csv(r"C:\Users\ehug0006\emu\aust-renewal\aust_renewal\data\aus_seroprev_data.csv")

In [ ]:
df_int = case_data.loc[case_data['Country'] == 'Australia']

df_int ['Date_reported'] = pd.to_datetime(df_int['Date_reported'], format="%d/%m/%Y")
df_int = df_int.set_index('Date_reported')

# drop uneeded columns
df_interp = df_int.drop(['Country','Country_code','WHO_region', 'Cumulative_cases','New_deaths','Cumulative_deaths'], axis=1)

df = df_interp.resample('W-SUN').interpolate(method='linear')

df['New_cases'] = df['New_cases'].fillna(0)
df = df.iloc[:,0]

In [ ]:
seroprev_data['date'] = pd.to_datetime(seroprev_data['date'])
seroprev_data = seroprev_data.set_index('date')
seroprev_data = seroprev_data.iloc[:,0]

In [ ]:
# Specify fixed parameters and get calibration data
proc_update_freq = 14
init_time = 50
data = df
pop = 26e6
analysis_start = datetime(2021, 12, 1)
analysis_end = datetime(2022, 10, 1)
init_start = analysis_start - timedelta(init_time)
init_end = analysis_start - timedelta(1)
select_data = data.loc[analysis_start: analysis_end]
init_data = data.resample("D").asfreq().interpolate().loc[init_start: init_end] / 7.0

In [ ]:
# Define model and fitter
proc_fitter = CosineMultiCurve()
renew_model = RenewalModel(pop, analysis_start, analysis_end, proc_update_freq, proc_fitter, GammaDens(), init_time, init_data, GammaDens())

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.TruncatedNormal(7.3, 0.5, low=1.0),
    "gen_sd": dist.TruncatedNormal(3.8, 0.5, low=1.0),
    "cdr": dist.Beta(15, 15), #(16,40)
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.TruncatedNormal(8, 0.5, low=1.0),
    "report_sd": dist.TruncatedNormal(3, 0.5, low=1.0),
}

In [ ]:
# Define calibration and calib data
calib_data = {
    "weekly_sum": select_data,
    "seropos": seroprev_data
             }
calib = StandardCalib(renew_model, priors, calib_data)

In [ ]:
# Run calibration
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=infer.init_to_uniform(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=2, num_samples=1000, num_warmup=1000)
mcmc.run(random.PRNGKey(1))

In [ ]:
# Grab sample of data from calibrated model outputs
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=800)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
# get model results and outputs
def get_full_result(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd):
    return renew_model.renewal_func(gen_mean, gen_sd, proc, cdr, rt_init, report_mean, report_sd)

full_wrap = jit(get_full_result)

In [ ]:
panel_subtitles = ['weekly_sum', 'serpos'] + PANEL_SUBTITLES[2:]
spaghetti = get_spaghetti_from_params(renew_model, sample_params, full_wrap, outputs=panel_subtitles)
quantiles_df = get_quant_df_from_spaghetti(renew_model, spaghetti, quantiles=[0.05, 0.5, 0.95], outputs=panel_subtitles)

In [ ]:
plot_uncertainty_patches(quantiles_df, select_data, qual_colours.Plotly, panel_subtitles).update_layout(showlegend=False)

In [ ]:
az.summary(idata)

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);